https://www.tensorflow.org/tutorials/structured_data/feature_columns

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import os
import seaborn as sns
%matplotlib inline

import pywt
import scipy
from scipy import signal
from scipy.signal import butter, deconvolve


Bad key "text.kerning_factor" on line 4 in
C:\Users\Ayman\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
test_path = os.path.join('data','test.csv')
train_path = os.path.join('data','train.csv')

test_df = pd.read_csv(test_path)
train_df = pd.read_csv(train_path)

In [3]:
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise_signal(x, wavelet='db4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * maddest(coeff[-level])

    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

    return pywt.waverec(coeff, wavelet, mode='per')

In [4]:
time_train = train_df.time
signal_train_denoised = denoise_signal(train_df.signal)
open_channels_train = train_df.open_channels

data = {
    'time': time_train,
    'signal_denoised': signal_train_denoised,
    'open_channels_train': open_channels_train
       }

train_data = pd.DataFrame(data=data)

In [5]:
from sklearn.model_selection import train_test_split

X = train_data[['time','signal_denoised']]
y = train_data.open_channels_train.values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
def deep_learning_model(X, y, inputs=2, epochs=10, outputs=11):
    import tensorflow as tf
    import datetime
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler, LabelEncoder
    from tensorflow.keras.losses import sparse_categorical_crossentropy
    from tensorflow.keras.utils import to_categorical

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_scaler = MinMaxScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    label_encoder = LabelEncoder()
    label_encoder.fit(y_train)
    encoded_y_train = label_encoder.transform(y_train)
    encoded_y_test = label_encoder.transform(y_test)

    y_train_categorical = to_categorical(encoded_y_train)
    y_test_categorical = to_categorical(encoded_y_test)
    
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    
    model = Sequential()
    model.add(Dense(units=100, activation='relu', input_dim=inputs))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=300, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=outputs, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print(model.summary())
    
    model.fit(
        X_train_scaled,
        y_train_categorical,
        epochs=epochs,
        shuffle=True,
        verbose=2
    )
    
    
    print('\n----------------------------------------------------------------\n')
    
    model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
    print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")
    
    encoded_predictions = model.predict_classes(X_test_scaled[:100])
    prediction_labels = label_encoder.inverse_transform(encoded_predictions)

    return print(f'''
    Predicted classes: {list([i for i in prediction_labels])}
    Actual Labels: {[i[0] for i in y_test[:100]]}
    ''')

In [7]:
def deep_learning_model(X, y, inputs=2, epochs=10, outputs=11):
    import tensorflow as tf
    import datetime
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler, LabelEncoder
    from tensorflow.keras.losses import sparse_categorical_crossentropy
    from tensorflow.keras.utils import to_categorical

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_scaler = MinMaxScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    label_encoder = LabelEncoder()
    label_encoder.fit(y_train)
    encoded_y_train = label_encoder.transform(y_train)
    encoded_y_test = label_encoder.transform(y_test)

    y_train_categorical = to_categorical(encoded_y_train)
    y_test_categorical = to_categorical(encoded_y_test)
    
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    
    model = Sequential()
    model.add(Dense(units=100, activation='relu', input_dim=inputs))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=300, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=50, activation='relu'))
    model.add(Dense(units=10, activation='relu'))
    model.add(Dense(units=outputs, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


    print(model.summary())
    
    model.fit(
        X_train_scaled,
        y_train_categorical,
        epochs=epochs,
        batch_size = 200,
        shuffle=True,
        verbose=1
    )
    
    
    print('\n----------------------------------------------------------------\n')
    
    model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
    print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")
    
    encoded_predictions = model.predict_classes(X_test_scaled[:100])
    prediction_labels = label_encoder.inverse_transform(encoded_predictions)

    return print(f'''
    Predicted classes: {list([i for i in prediction_labels])}
    Actual Labels: {[i[0] for i in y_test[:100]]}
    ''')

In [8]:
X = train_data[['time','signal_denoised']]
y = train_data.open_channels_train.values.reshape(-1,1)
q = deep_learning_model(X=X,y=y ,inputs=2,epochs = 10)
q

C:\Users\Ayman\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               300       
_________________________________________________________________
dense_1 (Dense)              (None, 200)               20200     
_________________________________________________________________
dense_2 (Dense)              (None, 300)               60300     
_________________________________________________________________
dense_3 (Dense)              (None, 200)               60200     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5

C:\Users\Ayman\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
